# 2.3 自动求梯度

In [1]:
import torch 
import numpy as np 

## 2.3.2 Tensor

In [2]:
x = torch.ones(2,2, requires_grad = True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [6]:
print(x.grad_fn)

None


In [7]:
y = x+2 
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [10]:
"""
每个Tensor都有一个.grad_fn属性，该属性即创建该Tensor的Function,
就是说该Tensor是不是通过某些运算得到的，
若是，则grad_fn返回一个与这些运算相关的对象，否则是None。
"""

# 像x这种直接创建的称为叶子节点，叶子节点对应的grad_fn是None
x.is_leaf

True

In [9]:
y.is_leaf

False

In [11]:
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [12]:
z = y*y*3 
z

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)

In [13]:
out = z.mean()
out

tensor(27., grad_fn=<MeanBackward0>)

In [14]:
# 通过.requires_grad_()来用in-place的方式改变requires_grad属性：

a = torch.randn(2,2)
a = ((a*3)/(a-1))
a

tensor([[ 0.4525, -2.3580],
        [ 1.8048,  0.7981]])

In [15]:
a.requires_grad

False

In [16]:
a.requires_grad_(True)

tensor([[ 0.4525, -2.3580],
        [ 1.8048,  0.7981]], requires_grad=True)

In [17]:
b = (a*a).sum()
b

tensor(9.6594, grad_fn=<SumBackward0>)

In [18]:
print(b.grad_fn)

## 2.3.3 梯度

In [19]:
out 
"""
因为out是一个标量，所以调用backward()时不需要指定求导变量：
"""

tensor(27., grad_fn=<MeanBackward0>)

In [20]:
out.backward()
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

In [21]:
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [ ]:
"""
注意：grad在反向传播过程中是累加的(accumulated)
，这意味着每一次运行反向传播，梯度都会累加之前的梯度，
所以一般在反向传播之前需把梯度清零。
"""

In [22]:
out2 = x.sum()
out2

tensor(4., grad_fn=<SumBackward0>)

In [23]:
out2.backward()
x.grad

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])

In [24]:
out3 = x.sum()
x.grad.data.zero_()
out3.backward()
x.grad

tensor([[1., 1.],
        [1., 1.]])

In [30]:
x = torch.tensor([1.0,2.0,3.0,4.0], requires_grad = True)
y = 2*x 
z = y.view(2,2)
z


tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)

In [31]:
"""
现在 z 不是一个标量，所以在调用backward时需要传入一个和
z同形的权重向量进行加权求和得到一个标量。
"""
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)

In [32]:
z.backward(v)
x.grad
# 注意，x.grad是和x同形的张量。

tensor([2.0000, 0.2000, 0.0200, 0.0020])

In [33]:
"""
中断梯度追踪的例子：
"""

x = torch.tensor(1.0, requires_grad= True)
y1 = x**2 

with torch.no_grad():
    y2 = x**3 
    
y3 = y1+y2 

In [34]:
x.requires_grad

True

In [35]:
y1, y1.requires_grad

(tensor(1., grad_fn=<PowBackward0>), True)

In [36]:
y2, y2.requires_grad

(tensor(1.), False)

In [37]:
y3, y3.requires_grad

(tensor(2., grad_fn=<AddBackward0>), True)

In [38]:
y3.backward()
x.grad

tensor(2.)

In [ ]:
"""
如果我们想要修改tensor的数值，
但是又不希望被autograd记录（即不会影响反向传播），
那么我么可以对tensor.data进行操作。
"""

In [39]:
x = torch.ones(1, requires_grad = True)
x

tensor([1.], requires_grad=True)

In [40]:
x.data

tensor([1.])

In [41]:
x.item()

1.0

In [42]:
x.data.requires_grad  # # 但是已经是独立于计算图之外

False

In [43]:
y= 2*x 
x.data *=100  # 只改变了值，不会记录在计算图，所以不会影响梯度传播

In [44]:
y.backward()
x

tensor([100.], requires_grad=True)

In [45]:
x.grad

tensor([2.])